<a href="https://colab.research.google.com/github/inshra12/ESM-MLP-baseline-protein-function/blob/main/notebooks/sequence-based-function-prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model 1

## Importing files

In [47]:
from data_loader import load_data,encode_labels

In [ ]:
!pip install fair-esm

In [49]:
from esm_embedder import save_embeddings

## Loading Data

In [ ]:
df=load_data("/content/raw_sequence_data.csv")

In [51]:
#save_embeddings("/content/raw_sequence_data.csv", "cls_embeddings.csv")

In [52]:
import pandas as pd
# Load ESM embeddings (previously extracted)
esm_df = pd.read_csv("/content/cls_embeddings.csv")  # must have column 'ID'

## Encoding multi-hot labels

In [53]:
encoded_labels, mlb = encode_labels(df["Labels"])

In [54]:
# Step 2: Convert encoded labels into a DataFrame
encoded_labels_df = pd.DataFrame(encoded_labels, columns=mlb.classes_)
encoded_labels_df["ID"] = df["ID"].values  # Add ID for merging

## Merge sequence embedding and labels in one file

In [55]:
# Step 3: Merge ESM embeddings (esm_df) with labels
final_df = pd.merge(esm_df, encoded_labels_df, on="ID")

In [58]:
# Move "ID" column to the front
cols = list(final_df.columns)
cols.remove("ID")
new_order = ["ID"] + cols
final_df = final_df[new_order]


In [ ]:
# Step 4: Save final_df to CSV
final_df.to_csv("esm_with_multihot_labels.csv", index=False)
print("Saved: esm_with_multihot_labels.csv")

## Splitting Column in X(Sequence) and Y(labels)

In [63]:
# Get all column names
all_columns = final_df.columns

# GO columns start from index 1281 onwards
go_columns = all_columns[1281:]

# Now define X and Y
X = final_df.drop(columns=["ID"] + list(go_columns))  # Features (1280 ESM cols)
Y = final_df[list(go_columns)]                        # Labels (GO terms)


In [66]:
X.to_csv("X_esm.csv", index=False)
Y.to_csv("Y_go.csv", index=False)